In [ ]:
import os
from pathlib import Path
import pandas as pd
import csv

## Settings

In [ ]:
SOURCE_DIRECTORY = "sample_data"
LABELS = "labels.csv"

## Import data

In [ ]:
df_labels = pd.read_csv(Path(SOURCE_DIRECTORY + r"/" + LABELS), names=["file_name", "label"])
df_labels.head()

In [ ]:
file_names = os.listdir(SOURCE_DIRECTORY)